In [1]:
import scipy.linalg as spl

from pennylane import numpy as np

from qbmqsp.rel_ent import relative_entropy
from gen_data import basis_encoding, gen_boltzmann_dist

In [2]:
def loss_func(ρ_data: np.ndarray[float], ρ_model: np.ndarray[float], pure: bool = False) -> float:
    if pure:
        return - np.trace(ρ_data @ spl.logm(ρ_model))
    return np.trace(ρ_data @  spl.logm(ρ_data)) - np.trace(ρ_data @ spl.logm(ρ_model))

In [3]:
n_qubits = 3
β = 1.0
f_boltzmann = gen_boltzmann_dist(n_qubits, β)
ρ_data = basis_encoding(f_boltzmann)

H = np.random.uniform(-1, 1, (2**n_qubits, 2**n_qubits)) + 1.j * np.random.uniform(-1, 1, (2**n_qubits, 2**n_qubits))
H = H + H.conj().T
expH = spl.expm(-β*H)
ρ = expH / np.trace(expH)
del expH

In [4]:
relative_entropy(ρ_data, ρ, check_state=True)

tensor(3.92133284, requires_grad=True)

In [5]:
loss_func(ρ_data, ρ, pure=True)

(3.9213328441027873-3.3306690738754696e-15j)

In [6]:
# Should be inf, if ρ_data is pure
relative_entropy(ρ, ρ_data, check_state=True)

/Users/danielortmann/miniconda3/envs/qbmqsp/lib/python3.11/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: divide by zero encountered in log
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)


tensor(inf, requires_grad=True)

In [7]:
# Should be inf, if ρ_data is pure, but a naive implementation result in a different value due to numerical issues of log(0)
loss_func(ρ, ρ_data, pure=False)

/Users/danielortmann/miniconda3/envs/qbmqsp/lib/python3.11/site-packages/scipy/linalg/_matfuncs.py:203: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


(44.87759034958813+2.097535291708984j)